In [1]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hys
import tkinter.filedialog as tkf
from scipy import optimize

In [2]:
# refer to https://scipy-cookbook.readthedocs.io/items/FittingData.html

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape) # row, col
    x = (X*data).sum()/total # row
    y = (Y*data).sum()/total # col
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [3]:
# refer to github.com/mkolopanis/python/blob/master/radialProfile.py

def radial_variance(image, profile_resolution=1.0, cbox_edge=None):
   
    y, x = np.indices(image.shape) * profile_resolution
    if not cbox_edge:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
        
    else:
        cbox_outy = int(image.shape[0]/2 - cbox_edge/2)
        cbox_outx = int(image.shape[1]/2 - cbox_edge/2)
        center_box = image[cbox_outy:-cbox_outy, cbox_outx:-cbox_outx]
        fit_params = fitgaussian(center_box)
        (_, center_y, center_x, _, _) = fit_params
        center = [int(round(center_y)+cbox_outy), int(round(center_x)+cbox_outx)]
        
    r = np.hypot(y - center[0]*profile_resolution, x - center[1]*profile_resolution)
    #plt.imshow(r, cmap="Accent")
    #plt.show()

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    #print(nr)
    
    csim = np.cumsum(i_sorted, dtype=float)
    sq_csim = np.cumsum(np.square(i_sorted), dtype=float)
    
    radial_avg = (csim[rind[1:]] - csim[rind[:-1]]) / nr
    square_avg = (sq_csim[rind[1:]] - sq_csim[rind[:-1]]) / nr
    
    radial_var = square_avg - np.square(radial_avg)
    return radial_avg, square_avg, radial_var    

In [ ]:
# refer to "github.com/mkolopanis/python/blob/master/radialProfile.py"

def radial_average(image, profile_resolution=1.0, cbox_edge=None):
   
    y, x = np.indices(image.shape) * profile_resolution
    if not cbox_edge:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
        
    else:
        cbox_outy = int(image.shape[0]/2 - cbox_edge/2)
        cbox_outx = int(image.shape[1]/2 - cbox_edge/2)
        center_box = image[cbox_outy:-cbox_outy, cbox_outx:-cbox_outx]
        fit_params = fitgaussian(center_box)
        (_, center_y, center_x, _, _) = fit_params
        center = [int(round(center_y)+cbox_outy), int(round(center_x)+cbox_outx)]
        
    r = np.hypot(y - center[0]*profile_resolution, x - center[1]*profile_resolution)
    #plt.imshow(r, cmap="Accent")
    #plt.show()

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    #print(nr)
    
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

In [ ]:
img = hys.load(tkf.askopenfilename()).data
print(img.shape)

In [ ]:
radial_avg, square_avg, radial_var = radial_variance(img, profile_resolution=5.0, cbox_edge=20)
plt.plot(np.square(radial_avg))
plt.plot(square_avg)
plt.show()

In [ ]:
plt.plot(radial_var)

In [ ]:
plt.plot(byint)
plt.plot(notbyint)
plt.show()

In [ ]:
y, x = np.indices(img.shape) * 1.5
print(y)
print(x)

In [ ]:
center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
print(center)

In [ ]:
r = np.hypot(x-center[0], y-center[1])
print(r.shape)
print(r)

In [ ]:
ind = np.argsort(r.flat)
print(ind.shape)
print(ind)

In [ ]:
r_sorted = r.flat[ind]
i_sorted = img.flat[ind]

In [ ]:
r_int = r_sorted.astype(int)
print(r_sorted[:50])
print(r_int[:50])

In [ ]:
deltar = r_int[1:] - r_int[:-1]
print(deltar[:10])

In [ ]:
deltar = r_sorted[1:] - r_sorted[:-1]
print(deltar[:10])

In [ ]:
rind = np.where(deltar>0.15)[0]
print(rind.shape)
print(rind)

In [ ]:
rind = np.where(deltar)[0]
print(rind.shape)
print(rind)

In [ ]:
nr = rind[1:] - rind[:-1]
print(nr)

In [ ]:
csim = np.cumsum(i_sorted, dtype=float)
print(csim.shape)
print(csim[:10])

In [ ]:
tbin = csim[rind[1:]] - csim[rind[:-1]]
print(tbin.shape)

In [ ]:
radial_profile = tbin / nr
print(radial_profile.shape)
print(radial_profile)